In [1]:
import pandas as pd
import re
import random
import sys
import re

import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize, sent_tokenize
import itertools
from collections import defaultdict

import numpy as np

import pickle

from __future__ import division

In [2]:
# reading data
sample = pd.read_csv("data/tweets.csv")
sample.head()

,PaxText,Reply
0,"@AirCanada thanks, what's te difference from t...","@Agent_613 Hi Bryan, Cargo may or may not be a..."
1,@SouthwestAir Tried cancel reservation w/i 24 ...,@mjij3 Hi there! Please follow/DM us your trav...
2,@SouthwestAir Can't DM without a follow :),@BobMcKelvey Whoops!
3,@SouthwestAir if my flight (4898) is delayed y...,@sozymicmike Fair! We're working hard to get y...
4,@AirCanada Just got a call similar to those We...,"@kerryinyyz Hi Kerry, these are certainly not ..."


In [3]:
# groupby multiple replies
sample = sample.groupby('PaxText')['Reply'].apply(lambda x: "%s" % ' '.join(x)).reset_index()

In [4]:
contractions = { 
    "i ain't": "am not",
    "you ain't": "you are not",
    "they ain't": "they are not",
    "she ain't": "she is not",
    "he ain't": "he is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"   
}

contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
def expand_contractions(s, contractions_dict=contractions, contractions_re=contractions_re):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [5]:
name_list = pd.read_csv("data/firstnames.csv")["firstname"].str.lower().tolist()
print name_list[:15]

def clean_text(text, name_list=name_list):
    # to lower case
    text = text.lower()
    # remove @ mentions
    text = re.sub(r'@\w+\b', '', text)
    # remove url links
    text = re.sub(r'\bhttp.+\b', '', text)
    # remove line break
    text = re.sub(r'\n', ' ', text)
    
    # expand contraction
    text = expand_contractions(text)
    
    # replace some common shorthands
    text = re.sub(r'&amp;', ' and ', text)
    text = re.sub(r'\bb/c\b', 'because', text)
    text = re.sub(r'\b&lt;\b', '<', text)
    text = re.sub(r'\b&gt;\b', '>', text)
    
    # remove punctuation
    text = re.sub(r'[^\w\d_\s]+', '', text)
    
    # get rid of common english names
    text = [w for w in text.split() if w not in name_list]
    
    # get rid of initials at the end
    if len(text) > 0:
        if re.match(r'\b(?:\/|\^)[A-z]{2}\b', text[-1]) is not None:
            text.pop()
        
    if len(text) <= 3:
        return "TO_DELETE"
    else:
        return " ".join(text)

['aaron', 'aaron', 'abbey', 'abbie', 'abby', 'abdul', 'abe', 'abel', 'abigail', 'abraham', 'abram', 'ada', 'adah', 'adalberto', 'adaline']


In [6]:
sample["PaxText"] = sample["PaxText"].apply(lambda x: clean_text(x))
sample["Reply"] = sample["Reply"].apply(lambda x: clean_text(x))
sample = sample[(sample["PaxText"] != "TO_DELETE") & (sample["Reply"] != "TO_DELETE")]
sample.head()

,PaxText,Reply
0,boeing747 sunset beautiful ht,hi glad you are having such a great day
2,happy 30th birthday emirates30 halamadrid,thank you emirates30 halamadrid
4,traveling firstclass is always style thank you...,we you too it is our pleasure having you on board
6,and having bag ripped but the baggage claim pp...,we are glad to hear our airport team took good...
8,off to la la land la wheelsup,sweet pic margeaux we luv that view vp


In [7]:
print len(sample)

845997


In [8]:
limit = {'maxq' : 500,
         'minq' : 3,
         'maxa' : 500,
         'mina' : 3
        }

UNK = 'unk'
VOCAB_SIZE = 6000

In [9]:
qlines = sample["PaxText"].tolist()
alines = sample["Reply"].tolist()

qtokenized = [sent.split(' ') for sent in qlines]
atokenized = [sent.split(' ') for sent in alines]

print "Q: " + " ".join(qtokenized[5])
print "A: " + " ".join(atokenized[5])

Q: leavin on a plane do not know when i be back again
A: have the best trip ever


In [10]:
def index_(tokenized_sentences, vocab_size):
    # get frequency distribution
    freq_dist = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    # get vocabulary of 'vocab_size' most used words
    vocab = freq_dist.most_common(vocab_size)
    # index2word
    index2word = ['_'] + [UNK] + [ x[0] for x in vocab ]
    # word2index
    word2index = dict([(w,i) for i,w in enumerate(index2word)] )
    return index2word, word2index, freq_dist

idx2w, w2idx, freq_dist = index_(qtokenized + atokenized, vocab_size=VOCAB_SIZE)

In [11]:
def pad_seq(seq, lookup, maxlen):
    indices = []
    for word in seq:
        if word in lookup:
            indices.append(lookup[word])
        else:
            indices.append(lookup[UNK])
    return indices + [0]*(maxlen - len(seq))

def zero_pad(qtokenized, atokenized, w2idx):
    # num of rows
    data_len = len(qtokenized)

    # numpy arrays to store indices
    idx_q = np.zeros([data_len, limit['maxq']], dtype=np.int32)
    idx_a = np.zeros([data_len, limit['maxa']], dtype=np.int32)

    for i in range(data_len):
        q_indices = pad_seq(qtokenized[i], w2idx, limit['maxq'])
        a_indices = pad_seq(atokenized[i], w2idx, limit['maxa'])

        #print(len(idx_q[i]), len(q_indices))
        #print(len(idx_a[i]), len(a_indices))
        idx_q[i] = np.array(q_indices)
        idx_a[i] = np.array(a_indices)

    return idx_q, idx_a

idx_q, idx_a = zero_pad(qtokenized, atokenized, w2idx)

In [12]:
# count of unknowns
unk_count = (idx_q == 1).sum() + (idx_a == 1).sum()
# count of words

word_count = (idx_q > 1).sum() + (idx_a > 1).sum()
# % unknown

print unk_count
print word_count
print '% unknown : {}'.format(100 * (unk_count/word_count))

1031451
27147884
% unknown : 3.79937898659


In [13]:
print len(idx_q)
print len(idx_a)

845997
845997


## Training a Model

In [14]:
idx_q_filtered = []
idx_a_filtered = []

for i in range(len(idx_q)):
    w_count_q = (idx_q[i] > 1).sum()
    u_count_q = (idx_q[i] == 1).sum()
    
    w_count_a = (idx_a[i] > 1).sum()
    u_count_a = (idx_a[i] == 1).sum()
    
    if w_count_q/(w_count_q + u_count_q) >= 0.8 and w_count_a/(w_count_a + u_count_a) >= 0.8:
        idx_q_filtered.append(np.array(idx_q[i]))
        idx_a_filtered.append(np.array(idx_a[i]))
        
idx_q_filtered = np.array(idx_q_filtered)
idx_a_filtered = np.array(idx_a_filtered)

print len(idx_q_filtered)
print len(idx_a_filtered)

796449
796449


In [15]:
def split_dataset(x, y, ratio = [0.7, 0.15, 0.15] ):
    # number of examples
    data_len = len(x)
    lens = [ int(data_len*item) for item in ratio ]

    trainX, trainY = x[:lens[0]], y[:lens[0]]
    testX, testY = x[lens[0]:lens[0]+lens[1]], y[lens[0]:lens[0]+lens[1]]
    validX, validY = x[-lens[-1]:], y[-lens[-1]:]

    return (trainX,trainY), (testX,testY), (validX,validY)

(trainX, trainY), (testX, testY), (validX, validY) = split_dataset(idx_q_filtered, idx_a_filtered)

trainX = trainX.tolist()
trainY = trainY.tolist()
testX = testX.tolist()
testY = testY.tolist()
validX = validX.tolist()
validY = validY.tolist()

print trainX[0]
print trainY[0]

[449, 2105, 11, 145, 2068, 59, 3, 6, 1, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *

In [17]:
trainX = tl.prepro.remove_pad_sequences(trainX)
trainY = tl.prepro.remove_pad_sequences(trainY)
testX = tl.prepro.remove_pad_sequences(testX)
testY = tl.prepro.remove_pad_sequences(testY)
validX = tl.prepro.remove_pad_sequences(validX)
validY = tl.prepro.remove_pad_sequences(validY)

print trainX[10]
print trainY[10]

[1, 2, 682, 765, 8, 27, 226, 78, 4685, 8, 38]
[134, 54, 964, 59, 3, 6, 81, 18, 25, 539]


In [18]:
###============= parameters
xseq_len = len(trainX)#.shape[-1]
yseq_len = len(trainY)#.shape[-1]
assert xseq_len == yseq_len

batch_size = 10
n_step = int(xseq_len/batch_size)

xvocab_size = len(idx2w)
emb_dim = 1024

In [19]:
print xseq_len
print n_step
print xvocab_size

557514
55751
6002


In [20]:
unk_id = w2idx['unk']   # 1
pad_id = w2idx['_']     # 0

start_id = xvocab_size   # 6002
end_id = xvocab_size + 1  # 6003

In [21]:
w2idx.update({'start_id': start_id})
w2idx.update({'end_id': end_id})
idx2w = idx2w + ['start_id', 'end_id']

xvocab_size = yvocab_size = xvocab_size + 2

# let us now save the necessary dictionaries
metadata = {
            'w2idx' : w2idx,
            'idx2w' : idx2w,
            'xvocab_size' : xvocab_size,
            'emb_dim': emb_dim
            }

# write to disk : data control dictionaries
with open('metadata_1M.pkl', 'wb') as f:
        pickle.dump(metadata, f)

In [22]:
print "encode_seqs:", [idx2w[id] for id in trainX[10]]

target_seqs = tl.prepro.sequences_add_end_id([trainY[10]], end_id=end_id)[0]
print "target_seqs:", [idx2w[id] for id in target_seqs]

decode_seqs = tl.prepro.sequences_add_start_id([trainY[10]], start_id=start_id, remove_last=False)[0]
print "decode_seqs:", [idx2w[id] for id in decode_seqs]

target_mask = tl.prepro.sequences_get_mask([target_seqs])[0]
print "target_mask:", target_mask

print(len(target_seqs), len(decode_seqs), len(target_mask))

encode_seqs: ['unk', 'to', 'nyc', 'miami', 'i', 'be', 'right', 'back', 'skypriority', 'i', 'am']
target_seqs: ['hello', 'great', 'pic', 'thank', 'you', 'for', 'flying', 'with', 'us', 'rd', 'end_id']
decode_seqs: ['start_id', 'hello', 'great', 'pic', 'thank', 'you', 'for', 'flying', 'with', 'us', 'rd']
target_mask: [1 1 1 1 1 1 1 1 1 1 1]
(11, 11, 11)


#### Model

In [23]:
def model(encode_seqs, decode_seqs, is_train=True, reuse=False):
    with tf.variable_scope("model", reuse=reuse):
        # for chatbot, you can use the same embedding layer,
        # for translation, you may want to use 2 separated embedding layers
        with tf.variable_scope("embedding") as vs:
            net_encode = EmbeddingInputlayer(
                inputs = encode_seqs,
                vocabulary_size = xvocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
            
            vs.reuse_variables()
            tl.layers.set_name_reuse(True)
            
            net_decode = EmbeddingInputlayer(
                inputs = decode_seqs,
                vocabulary_size = xvocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
            
        net_rnn = Seq2Seq(net_encode, net_decode,
                cell_fn = tf.contrib.rnn.BasicLSTMCell,
                n_hidden = emb_dim,
                initializer = tf.random_uniform_initializer(-0.1, 0.1),
                encode_sequence_length = retrieve_seq_length_op2(encode_seqs),
                decode_sequence_length = retrieve_seq_length_op2(decode_seqs),
                initial_state_encode = None,
                dropout = (0.5 if is_train else None),
                n_layer = 3,
                return_seq_2d = True,
                name = 'seq2seq')
        
        net_out = DenseLayer(net_rnn, n_units=xvocab_size, act=tf.identity, name='output')
    return net_out, net_rnn

In [24]:
# model for training
encode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs")
decode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs")
target_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs")
target_mask = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") # tl.prepro.sequences_get_mask()
net_out, _ = model(encode_seqs, decode_seqs, is_train=True, reuse=False)

  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (6004, 1024)
  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (6004, 1024)
  [**] Seq2Seq model/seq2seq: n_hidden:1024 cell_fn:BasicLSTMCell dropout:0.5 n_layer:3
  [TL] DynamicRNNLayer model/seq2seq/seq2seq_encode: n_hidden:1024, in_dim:3 in_shape:(10, ?, 1024) cell_fn:BasicLSTMCell dropout:0.5 n_layer:3
       batch_size (concurrent processes): 10
  [TL] DynamicRNNLayer model/seq2seq/seq2seq_decode: n_hidden:1024, in_dim:3 in_shape:(10, ?, 1024) cell_fn:BasicLSTMCell dropout:0.5 n_layer:3
       batch_size (concurrent processes): 10
  [TL] DenseLayer  model/output: 6004 identity


In [25]:
# loss for training
loss = tl.cost.cross_entropy_seq_with_mask(logits=net_out.outputs, 
                                           target_seqs=target_seqs, 
                                           input_mask=target_mask, 
                                           return_details=False, 
                                           name='cost')

In [26]:
net_out.print_params(False)

lr = 0.0001
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

  param   0: model/embedding/seq_embedding/embeddings:0 (6004, 1024)       float32_ref
  param   1: model/seq2seq/seq2seq_decode/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0 (2048, 4096)       float32_ref
  param   2: model/seq2seq/seq2seq_decode/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0 (4096,)            float32_ref
  param   3: model/seq2seq/seq2seq_decode/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0 (2048, 4096)       float32_ref
  param   4: model/seq2seq/seq2seq_decode/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0 (4096,)            float32_ref
  param   5: model/seq2seq/seq2seq_decode/rnn/multi_rnn_cell/cell_2/basic_lstm_cell/kernel:0 (2048, 4096)       float32_ref
  param   6: model/seq2seq/seq2seq_decode/rnn/multi_rnn_cell/cell_2/basic_lstm_cell/bias:0 (4096,)            float32_ref
  param   7: model/seq2seq/seq2seq_encode/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0 (2048, 4096)       float32_ref
  param   8: model/seq2seq/seq2seq_encode/rnn/multi

In [27]:
# model for inferencing
encode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="encode_seqs")
decode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="decode_seqs")
net, net_rnn = model(encode_seqs2, decode_seqs2, is_train=False, reuse=True)
y = tf.nn.softmax(net.outputs)

  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (6004, 1024)
  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (6004, 1024)
  [**] Seq2Seq model/seq2seq: n_hidden:1024 cell_fn:BasicLSTMCell dropout:None n_layer:3
  [TL] DynamicRNNLayer model/seq2seq/seq2seq_encode: n_hidden:1024, in_dim:3 in_shape:(1, ?, 1024) cell_fn:BasicLSTMCell dropout:None n_layer:3
       batch_size (concurrent processes): 1
  [TL] DynamicRNNLayer model/seq2seq/seq2seq_decode: n_hidden:1024, in_dim:3 in_shape:(1, ?, 1024) cell_fn:BasicLSTMCell dropout:None n_layer:3
       batch_size (concurrent processes): 1
  [TL] DenseLayer  model/output: 6004 identity


#### Train

In [28]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess)

n_epoch = 50

for epoch in range(n_epoch):
    epoch_time = time.time()
    ## shuffle training data
    from sklearn.utils import shuffle
    trainX, trainY = shuffle(trainX, trainY, random_state=0)
    ## train an epoch
    total_err, n_iter = 0, 0
    for X, Y in tl.iterate.minibatches(inputs=trainX, targets=trainY, batch_size=batch_size, shuffle=False):
        step_time = time.time()

        X = tl.prepro.pad_sequences(X)
        _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
        _target_seqs = tl.prepro.pad_sequences(_target_seqs)

        _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
        _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
        _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

        ## you can view the data here
        # for i in range(len(X)):
        #     print(i, [idx2w[id] for id in X[i]])
        #     print(i, [idx2w[id] for id in Y[i]])
        #     print(i, [idx2w[id] for id in _target_seqs[i]])
        #     print(i, [idx2w[id] for id in _decode_seqs[i]])
        #     print(i, _target_mask[i])
        #     print(len(_target_seqs[i]), len(_decode_seqs[i]), len(_target_mask[i]))
        # exit()

        _, err = sess.run([train_op, loss],
                        {encode_seqs: X,
                        decode_seqs: _decode_seqs,
                        target_seqs: _target_seqs,
                        target_mask: _target_mask})

        if n_iter % 200 == 0:
            print("Epoch[%d/%d] step:[%d/%d] loss:%f took:%.5fs" % (epoch, n_epoch, n_iter, n_step, err, time.time() - step_time))

        total_err += err; n_iter += 1

Epoch[0/50] step:[0/55751] loss:9.329982 took:0.66202s
Epoch[0/50] step:[200/55751] loss:6.587735 took:0.20728s
Epoch[0/50] step:[400/55751] loss:5.720950 took:0.17277s
Epoch[0/50] step:[600/55751] loss:6.133894 took:0.17288s
Epoch[0/50] step:[800/55751] loss:5.805511 took:0.22027s
Epoch[0/50] step:[1000/55751] loss:5.839880 took:0.24309s
Epoch[0/50] step:[1200/55751] loss:5.374759 took:0.17250s
Epoch[0/50] step:[1400/55751] loss:5.749952 took:0.20990s
Epoch[0/50] step:[1600/55751] loss:5.726033 took:0.21670s
Epoch[0/50] step:[1800/55751] loss:5.973401 took:0.44089s
Epoch[0/50] step:[2000/55751] loss:5.573874 took:0.23528s
Epoch[0/50] step:[2200/55751] loss:5.358806 took:0.22894s
Epoch[0/50] step:[2400/55751] loss:5.166379 took:0.24321s
Epoch[0/50] step:[2600/55751] loss:5.782174 took:0.26178s
Epoch[0/50] step:[2800/55751] loss:5.653172 took:0.19375s
Epoch[0/50] step:[3000/55751] loss:5.658379 took:0.27822s
Epoch[0/50] step:[3200/55751] loss:5.659799 took:0.17825s
Epoch[0/50] step:[340

Epoch[0/50] step:[28000/55751] loss:3.369488 took:0.23412s
Epoch[0/50] step:[28200/55751] loss:3.383142 took:0.20501s
Epoch[0/50] step:[28400/55751] loss:3.885865 took:0.18308s
Epoch[0/50] step:[28600/55751] loss:2.856195 took:0.32355s
Epoch[0/50] step:[28800/55751] loss:3.557212 took:0.18434s
Epoch[0/50] step:[29000/55751] loss:3.687995 took:0.24290s
Epoch[0/50] step:[29200/55751] loss:3.534324 took:0.30945s
Epoch[0/50] step:[29400/55751] loss:3.769305 took:0.27691s
Epoch[0/50] step:[29600/55751] loss:2.767051 took:0.21974s
Epoch[0/50] step:[29800/55751] loss:3.687118 took:0.27164s
Epoch[0/50] step:[30000/55751] loss:3.619653 took:0.23112s
Epoch[0/50] step:[30200/55751] loss:3.228127 took:0.21382s
Epoch[0/50] step:[30400/55751] loss:2.504023 took:0.20525s
Epoch[0/50] step:[30600/55751] loss:3.566951 took:0.24458s
Epoch[0/50] step:[30800/55751] loss:3.480915 took:0.35202s
Epoch[0/50] step:[31000/55751] loss:2.840909 took:0.18793s
Epoch[0/50] step:[31200/55751] loss:3.201037 took:0.2902

Epoch[1/50] step:[0/55751] loss:3.269706 took:0.22681s
Epoch[1/50] step:[200/55751] loss:3.157961 took:0.20133s
Epoch[1/50] step:[400/55751] loss:2.649162 took:0.44950s
Epoch[1/50] step:[600/55751] loss:2.935627 took:0.20092s
Epoch[1/50] step:[800/55751] loss:3.113456 took:0.35136s
Epoch[1/50] step:[1000/55751] loss:2.762359 took:0.32038s
Epoch[1/50] step:[1200/55751] loss:3.273232 took:0.20051s
Epoch[1/50] step:[1400/55751] loss:3.328963 took:0.24525s
Epoch[1/50] step:[1600/55751] loss:3.385464 took:0.25768s
Epoch[1/50] step:[1800/55751] loss:2.329816 took:0.24037s
Epoch[1/50] step:[2000/55751] loss:3.249865 took:0.29163s
Epoch[1/50] step:[2200/55751] loss:3.196080 took:0.24538s
Epoch[1/50] step:[2400/55751] loss:3.279532 took:0.26467s
Epoch[1/50] step:[2600/55751] loss:2.880117 took:0.19761s
Epoch[1/50] step:[2800/55751] loss:3.310514 took:0.21110s
Epoch[1/50] step:[3000/55751] loss:3.944660 took:0.24146s
Epoch[1/50] step:[3200/55751] loss:2.663777 took:0.26153s
Epoch[1/50] step:[340

Epoch[1/50] step:[28000/55751] loss:2.964138 took:0.21915s
Epoch[1/50] step:[28200/55751] loss:3.803422 took:0.29948s
Epoch[1/50] step:[28400/55751] loss:3.052292 took:0.21141s
Epoch[1/50] step:[28600/55751] loss:3.136235 took:0.19883s
Epoch[1/50] step:[28800/55751] loss:2.399521 took:0.18977s
Epoch[1/50] step:[29000/55751] loss:3.219660 took:0.36529s
Epoch[1/50] step:[29200/55751] loss:3.232952 took:0.17605s
Epoch[1/50] step:[29400/55751] loss:3.471229 took:0.21588s
Epoch[1/50] step:[29600/55751] loss:3.132441 took:0.25513s
Epoch[1/50] step:[29800/55751] loss:2.367842 took:0.23628s
Epoch[1/50] step:[30000/55751] loss:3.328022 took:0.29696s
Epoch[1/50] step:[30200/55751] loss:3.211452 took:0.25688s
Epoch[1/50] step:[30400/55751] loss:2.549711 took:0.20888s
Epoch[1/50] step:[30600/55751] loss:2.774108 took:0.30713s
Epoch[1/50] step:[30800/55751] loss:3.132149 took:0.20643s
Epoch[1/50] step:[31000/55751] loss:3.436945 took:0.27989s
Epoch[1/50] step:[31200/55751] loss:3.603184 took:0.2821

Epoch[2/50] step:[0/55751] loss:2.501693 took:0.28517s
Epoch[2/50] step:[200/55751] loss:3.264061 took:0.17999s
Epoch[2/50] step:[400/55751] loss:2.261023 took:0.18095s
Epoch[2/50] step:[600/55751] loss:2.295483 took:0.18325s
Epoch[2/50] step:[800/55751] loss:2.729541 took:0.30452s
Epoch[2/50] step:[1000/55751] loss:3.141888 took:0.20183s
Epoch[2/50] step:[1200/55751] loss:3.639694 took:0.24217s
Epoch[2/50] step:[1400/55751] loss:3.342757 took:0.23480s
Epoch[2/50] step:[1600/55751] loss:2.295555 took:0.25035s
Epoch[2/50] step:[1800/55751] loss:2.438114 took:0.40312s
Epoch[2/50] step:[2000/55751] loss:3.242840 took:0.31890s
Epoch[2/50] step:[2200/55751] loss:2.458366 took:0.26551s
Epoch[2/50] step:[2400/55751] loss:2.971375 took:0.25968s
Epoch[2/50] step:[2600/55751] loss:3.186669 took:0.20600s
Epoch[2/50] step:[2800/55751] loss:3.266727 took:0.30737s
Epoch[2/50] step:[3000/55751] loss:3.019815 took:0.29779s
Epoch[2/50] step:[3200/55751] loss:3.009851 took:0.22807s
Epoch[2/50] step:[340

...

Epoch[47/50] step:[53000/55751] loss:2.065729 took:0.19456s
Epoch[47/50] step:[53200/55751] loss:2.444023 took:0.30468s
Epoch[47/50] step:[53400/55751] loss:2.664458 took:0.25849s
Epoch[47/50] step:[53600/55751] loss:1.839886 took:0.29984s
Epoch[47/50] step:[53800/55751] loss:2.462192 took:0.22303s
Epoch[47/50] step:[54000/55751] loss:2.492886 took:0.18579s
Epoch[47/50] step:[54200/55751] loss:1.817048 took:0.38204s
Epoch[47/50] step:[54400/55751] loss:1.929944 took:0.18832s
Epoch[47/50] step:[54600/55751] loss:2.134052 took:0.22709s
Epoch[47/50] step:[54800/55751] loss:1.936413 took:0.23039s
Epoch[47/50] step:[55000/55751] loss:2.523053 took:0.19439s
Epoch[47/50] step:[55200/55751] loss:2.363389 took:0.23483s
Epoch[47/50] step:[55400/55751] loss:1.978359 took:0.20474s
Epoch[47/50] step:[55600/55751] loss:2.887799 took:0.44971s
Epoch[48/50] step:[0/55751] loss:1.923836 took:0.18861s
Epoch[48/50] step:[200/55751] loss:2.631896 took:0.22370s
Epoch[48/50] step:[400/55751] loss:1.772829 to

Epoch[48/50] step:[24800/55751] loss:2.720794 took:0.22603s
Epoch[48/50] step:[25000/55751] loss:1.725913 took:0.32832s
Epoch[48/50] step:[25200/55751] loss:1.935414 took:0.27912s
Epoch[48/50] step:[25400/55751] loss:1.891289 took:0.23663s
Epoch[48/50] step:[25600/55751] loss:2.355294 took:0.32782s
Epoch[48/50] step:[25800/55751] loss:2.363008 took:0.35423s
Epoch[48/50] step:[26000/55751] loss:1.961689 took:0.31092s
Epoch[48/50] step:[26200/55751] loss:1.687769 took:0.23782s
Epoch[48/50] step:[26400/55751] loss:2.420962 took:0.19594s
Epoch[48/50] step:[26600/55751] loss:1.642516 took:0.21945s
Epoch[48/50] step:[26800/55751] loss:2.550986 took:0.19625s
Epoch[48/50] step:[27000/55751] loss:2.051977 took:0.36339s
Epoch[48/50] step:[27200/55751] loss:2.535269 took:0.23821s
Epoch[48/50] step:[27400/55751] loss:2.626216 took:0.29358s
Epoch[48/50] step:[27600/55751] loss:2.200382 took:0.30248s
Epoch[48/50] step:[27800/55751] loss:2.616497 took:0.24235s
Epoch[48/50] step:[28000/55751] loss:2.4

Epoch[48/50] step:[52200/55751] loss:1.906689 took:0.19815s
Epoch[48/50] step:[52400/55751] loss:2.609971 took:0.20407s
Epoch[48/50] step:[52600/55751] loss:2.213368 took:0.20067s
Epoch[48/50] step:[52800/55751] loss:2.350420 took:0.18873s
Epoch[48/50] step:[53000/55751] loss:2.264428 took:0.22892s
Epoch[48/50] step:[53200/55751] loss:2.142832 took:0.35526s
Epoch[48/50] step:[53400/55751] loss:2.367175 took:0.24999s
Epoch[48/50] step:[53600/55751] loss:2.294425 took:0.34706s
Epoch[48/50] step:[53800/55751] loss:1.750660 took:0.17772s
Epoch[48/50] step:[54000/55751] loss:2.323182 took:0.24161s
Epoch[48/50] step:[54200/55751] loss:2.002518 took:0.21033s
Epoch[48/50] step:[54400/55751] loss:2.652279 took:0.33452s
Epoch[48/50] step:[54600/55751] loss:2.700330 took:0.25083s
Epoch[48/50] step:[54800/55751] loss:2.456713 took:0.26509s
Epoch[48/50] step:[55000/55751] loss:2.265565 took:0.30755s
Epoch[48/50] step:[55200/55751] loss:2.230322 took:0.18373s
Epoch[48/50] step:[55400/55751] loss:2.7

Epoch[49/50] step:[24000/55751] loss:2.025722 took:0.20553s
Epoch[49/50] step:[24200/55751] loss:2.719872 took:0.26143s
Epoch[49/50] step:[24400/55751] loss:1.764893 took:0.24655s
Epoch[49/50] step:[24600/55751] loss:2.043299 took:0.22124s
Epoch[49/50] step:[24800/55751] loss:2.095097 took:0.21556s
Epoch[49/50] step:[25000/55751] loss:1.852816 took:0.19264s
Epoch[49/50] step:[25200/55751] loss:2.254224 took:0.21574s
Epoch[49/50] step:[25400/55751] loss:2.887486 took:0.42318s
Epoch[49/50] step:[25600/55751] loss:2.139684 took:0.25997s
Epoch[49/50] step:[25800/55751] loss:2.754926 took:0.29533s
Epoch[49/50] step:[26000/55751] loss:1.624055 took:0.20740s
Epoch[49/50] step:[26200/55751] loss:2.828397 took:0.22959s
Epoch[49/50] step:[26400/55751] loss:3.272379 took:0.21587s
Epoch[49/50] step:[26600/55751] loss:2.536821 took:0.18318s
Epoch[49/50] step:[26800/55751] loss:2.569901 took:0.26595s
Epoch[49/50] step:[27000/55751] loss:2.451966 took:0.21422s
Epoch[49/50] step:[27200/55751] loss:2.3

Epoch[49/50] step:[51400/55751] loss:1.994718 took:0.20464s
Epoch[49/50] step:[51600/55751] loss:2.005978 took:0.28743s
Epoch[49/50] step:[51800/55751] loss:2.805684 took:0.26464s
Epoch[49/50] step:[52000/55751] loss:1.844638 took:0.25742s
Epoch[49/50] step:[52200/55751] loss:2.435005 took:0.20836s
Epoch[49/50] step:[52400/55751] loss:2.836753 took:0.32289s
Epoch[49/50] step:[52600/55751] loss:1.969807 took:0.22133s
Epoch[49/50] step:[52800/55751] loss:1.914384 took:0.19575s
Epoch[49/50] step:[53000/55751] loss:2.016619 took:0.37282s
Epoch[49/50] step:[53200/55751] loss:2.404088 took:0.36795s
Epoch[49/50] step:[53400/55751] loss:2.072329 took:0.21565s
Epoch[49/50] step:[53600/55751] loss:2.648263 took:0.28318s
Epoch[49/50] step:[53800/55751] loss:2.273766 took:0.19336s
Epoch[49/50] step:[54000/55751] loss:2.042050 took:0.17852s
Epoch[49/50] step:[54200/55751] loss:3.117425 took:0.22914s
Epoch[49/50] step:[54400/55751] loss:2.622095 took:0.18073s
Epoch[49/50] step:[54600/55751] loss:2.4

In [29]:
tl.files.save_npz(net.all_params, name='n_1M.npz', sess=sess)

[*] n_1M.npz saved
